In [1]:
import psycopg2
from psycopg2.extras import execute_values
import os
from dotenv import load_dotenv
# load from .env file.
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [22]:
class VectorDB():
    def __init__(self, db="mydb", user="postgres"):
        self.conn = psycopg2.connect(
          host=os.environ.get("PG_HOST"),
          database=os.environ.get("PG_NAME"),
          user=os.environ.get("PG_USER"),
          password=os.environ.get("PG_PASSWORD"))
        self.cur = self.conn.cursor()

    def query(self, query):
        self.cur.execute(query)

    def insert(self, records: list):
        try: 
            # self.cur.execute("INSERT INTO vectortb (id, file_name, image_vector) VALUES (%s, %s, %s)", (id, file_name, image_vector,))
            execute_values(self.cur, "INSERT INTO vectortb (file_name, image_vector) VALUES %s", records)
            self.conn.commit()
        except Exception as e:
            self.conn.rollback()
            print(e)

    def select(self, id):
        self.cur.execute("SELECT * FROM vectortb WHERE id = %s", (id,))
        return self.cur.fetchone()
    
    def select_all(self):
        self.cur.execute("SELECT * FROM vectortb")
        return self.cur.fetchall()
    
    def delete_all(self):
        self.cur.execute("DELETE FROM vectortb")
        self.conn.commit()
    
    def distance(self, image_vector, k=5):
        self.cur.execute("SELECT id, file_name, image_vector, (image_vector <-> %s) AS distance FROM vectortb ORDER BY distance LIMIT %s", (image_vector, k,))
        return self.cur.fetchall()
    
    def cosine_similarity(self, image_vector, k=5):
        self.cur.execute("SELECT id, file_name, image_vector, 1 - (image_vector <=> %s) AS similarity FROM vectortb ORDER BY similarity DESC LIMIT %s", (image_vector, k,))
        return self.cur.fetchall()

    def close(self):
        self.cur.close()
        self.conn.close()

    def __del__(self):
        self.close()

vector_db = VectorDB()

In [26]:
vector_db.delete_all()

records = []
for i in range(1, 5):
    records.append((f"{i}th file", ([i]*768).__repr__()))

# # print(records)
vector_db.insert(records)
vector_db.select_all()

[(6,
  '1th file',
  '[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1

In [111]:
vector_db.cosine_similarity('[-1,1]')

[(2, '2th file', '[2,1]', -0.316227766016838),
 (4, '4th file', '[4,1]', -0.514495755427526),
 (3, '3th file', '[3,-1]', -0.894427190999916),
 (1, '1th file', '[1,-1]', -1.0)]

In [103]:
vector_db.distance('[0, 0]', k=1)

[(1, '1th file', '[1,-1]', 1.4142135623731)]

In [17]:
([1] * 5).__repr__()


'[1, 1, 1, 1, 1]'